In [1]:
!ls

CNN_best_weights.01-0.7850.hdf5  import_pb_to_tensorboard.py
CNN_best_weights.01-0.7861.hdf5  output_cnn_glove_300.csv
CNN_best_weights.01-0.7873.hdf5  pix
Graph				 test_clean.csv
idsMatrix.npy			 train_clean.csv
idsMatrix_test-300.npy		 Untitled0.ipynb
idsMatrix_train-300.npy		 wordList-300.npy
idsMatrix_validation-300.npy	 wordVectors-300.npy


In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [3]:
csv = 'train_clean.csv'
my_df = pd.read_csv(csv)
my_df.head()

,id,text,target
0,86426,she should ask few native americans what their...,1
1,90194,go home you re drunk,1
2,16820,amazon is investigating chinese employees who ...,0
3,62688,someone should vetaken this piece of shit to v...,1
4,43605,obama wanted liberals amp illegals to move int...,0


In [4]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13239 entries, 0 to 13238
Data columns (total 3 columns):
id        13239 non-null int64
text      13239 non-null object
target    13239 non-null int64
dtypes: int64(2), object(1)
memory usage: 310.4+ KB


In [5]:
x = my_df.text
y = my_df.target


In [6]:
from sklearn.model_selection import train_test_split
SEED = 50
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.15, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.15, random_state=SEED)

In [7]:



print ("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print ("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print ("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))



Train set has total 11253 entries with 66.73% negative, 33.27% positive
Validation set has total 1688 entries with 67.00% negative, 33.00% positive
Test set has total 298 entries with 66.78% negative, 33.22% positive


In [8]:
wordsList = np.load('wordList-300.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors-300.npy')
print ('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


In [9]:
print(len(wordsList))
print(wordVectors.shape)

400000
(400000, 300)


In [10]:
import tensorflow as tf
maxSeqLength = 10 #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
firstSentence = np.zeros((maxSeqLength), dtype='int32')
firstSentence[0] = wordsList.index("i")
firstSentence[1] = wordsList.index("thought")
firstSentence[2] = wordsList.index("the")
firstSentence[3] = wordsList.index("movie")
firstSentence[4] = wordsList.index("was")
firstSentence[5] = wordsList.index("incredible")
firstSentence[6] = wordsList.index("and")
firstSentence[7] = wordsList.index("inspiring")
#firstSentence[8] and firstSentence[9] are going to be 0
print(firstSentence.shape)
print(firstSentence) #Shows the row index for each word

(10,)
[   41   804     0  1005    15  7446     5 13767     0     0]


In [11]:
#from os import listdir
#from os.path import isfile, join
#positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
#negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]
numWords = []
k = my_df.text
k = np.array(k)

for i in range(len(k)) :
  line = str(k[i])
  counter = len(line.split())
  numWords.append(counter)
  
  

#for pf in positiveFiles:
#    with open(pf, "r", encoding='utf-8') as f:
#        line=f.readline()
#        counter = len(line.split())
#        numWords.append(counter)       
#print('Positive files finished')

#for nf in negativeFiles:
#    with open(nf, "r", encoding='utf-8') as f:
#        line=f.readline()
#        counter = len(line.split())
#        numWords.append(counter)  
#print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))



The total number of files is 13239
The total number of words in the files is 244085
The average number of words in the files is 18.436815469446334


In [12]:
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,firstSentence).eval().shape)

(10, 300)


In [13]:
k = my_df.text
k = np.array(k)

In [14]:
maxSeqLength = 65

In [15]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [16]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0

for i in range(len(k)) :
  indexCounter = 0
  line = str(k[i])
  cleanedLine = cleanSentences(line)
  split = cleanedLine.split()
  for word in split:
    try :
      ids[fileCounter][indexCounter] = wordsList.index(word)
    except ValueError:
      ids[fileCounter][indexCounter] = 399999
      
    indexCounter = indexCounter + 1  
    if indexCounter >= maxSeqLength:
      break
  fileCounter = fileCounter + 1   
      
      
      

# for pf in positiveFiles:
#    with open(pf, "r") as f:
#        indexCounter = 0
#        line=f.readline()
#        cleanedLine = cleanSentences(line)
#        split = cleanedLine.split()
#        for word in split:
#            try:
#                ids[fileCounter][indexCounter] = wordsList.index(word)
#            except ValueError:
#                ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
#            indexCounter = indexCounter + 1
#            if indexCounter >= maxSeqLength:
#                break
#        fileCounter = fileCounter + 1 

# for nf in negativeFiles:
#    with open(nf, "r") as f:
#        indexCounter = 0
#        line=f.readline()
#        cleanedLine = cleanSentences(line)
#        split = cleanedLine.split()
#        for word in split:
#            try:
#                ids[fileCounter][indexCounter] = wordsList.index(word)
#            except ValueError:
#                ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
#            indexCounter = indexCounter + 1
#            if indexCounter >= maxSeqLength:
#                break
#        fileCounter = fileCounter + 1 
# #Pass into embedding function and see if it evaluates. 

# np.save('idsMatrix', ids)

In [17]:
np.save('idsMatrix', ids)

In [18]:
len(ids)

13239

In [19]:
ids[0]

array([  67,  189, 1712,  306, 1683,  827,  102,   44,  190,   13,   37,
         14,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [20]:
ids1 = np.load('idsMatrix.npy')

In [21]:
my_df.text[13238]

'vs and is'

In [22]:
ids[13238]

array([7091,    5,   14,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [23]:
my_df['seq'] = ids.tolist()

In [24]:
my_df

,id,text,target,seq
0,86426,she should ask few native americans what their...,1,"[67, 189, 1712, 306, 1683, 827, 102, 44, 190, ..."
1,90194,go home you re drunk,1,"[242, 163, 81, 1765, 8862, 0, 0, 0, 0, 0, 0, 0..."
2,16820,amazon is investigating chinese employees who ...,0,"[8427, 14, 3463, 327, 1242, 38, 32, 1514, 2169..."
3,62688,someone should vetaken this piece of shit to v...,1,"[1318, 189, 399999, 37, 2365, 3, 34855, 4, 855..."
4,43605,obama wanted liberals amp illegals to move int...,0,"[765, 741, 8193, 8457, 38273, 4, 483, 75, 639,..."
5,97670,liberals are all kookoo,1,"[8193, 32, 64, 399999, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,77444,oh noes tough shit,1,"[3202, 68385, 1595, 34855, 0, 0, 0, 0, 0, 0, 0..."
7,52415,was literally just talking about this lol all ...,1,"[15, 5917, 120, 1901, 59, 37, 73048, 64, 1329,..."
8,45157,buy more icecream,0,"[987, 56, 244867, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
9,13384,canada doesn need another cuck we already have...,1,"[774, 66019, 408, 170, 399999, 53, 411, 33, 57..."


In [25]:
x = my_df.text
y = my_df.target
#z = my_df.seq

In [26]:
from sklearn.model_selection import train_test_split
SEED = 50
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.15, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.15, random_state=SEED)

In [27]:
train_arr = np.array(x_train)

In [28]:
ids_train = np.zeros((len(train_arr), maxSeqLength), dtype='int32')
fileCounter = 0

for i in range(len(train_arr)) :
  indexCounter = 0
  line = str(train_arr[i])
  cleanedLine = cleanSentences(line)
  split = cleanedLine.split()
  for word in split:
    try :
      ids_train[fileCounter][indexCounter] = wordsList.index(word)
    except ValueError:
      ids_train[fileCounter][indexCounter] = 399999
      
    indexCounter = indexCounter + 1  
    if indexCounter >= maxSeqLength:
      break
  fileCounter = fileCounter + 1

In [29]:
len(ids_train)

11253

In [30]:
np.save('idsMatrix_train-300', ids_train)

In [31]:
val_arr = np.array(x_validation)

In [32]:
ids_val = np.zeros((len(val_arr), maxSeqLength), dtype='int32')
fileCounter = 0

for i in range(len(val_arr)) :
  indexCounter = 0
  line = str(val_arr[i])
  cleanedLine = cleanSentences(line)
  split = cleanedLine.split()
  for word in split:
    try :
      ids_val[fileCounter][indexCounter] = wordsList.index(word)
    except ValueError:
      ids_val[fileCounter][indexCounter] = 399999
      
    indexCounter = indexCounter + 1  
    if indexCounter >= maxSeqLength:
      break
  fileCounter = fileCounter + 1

In [33]:
np.save('idsMatrix_validation-300', ids_val)

In [34]:
test_arr = np.array(x_test)

In [35]:
ids_test = np.zeros((len(test_arr), maxSeqLength), dtype='int32')
fileCounter = 0

for i in range(len(test_arr)) :
  indexCounter = 0
  line = str(test_arr[i])
  cleanedLine = cleanSentences(line)
  split = cleanedLine.split()
  for word in split:
    try :
      ids_test[fileCounter][indexCounter] = wordsList.index(word)
    except ValueError:
      ids_test[fileCounter][indexCounter] = 399999
      
    indexCounter = indexCounter + 1  
    if indexCounter >= maxSeqLength:
      break
  fileCounter = fileCounter + 1

In [36]:
np.save('idsMatrix_test-300', ids_test)

In [37]:
x_train_seq = ids_train

In [38]:
x_train_seq[:5]

array([[ 56066,   5216,     53,    408,     81,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0],
       [ 27052,     19,   3518,  16015,     91,   8193,     17,   2563,
          3689,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,

In [39]:
x_train[:5]

8351                            maga patriots we need you
5351    cp as usual interviewing only liberals with ne...
7175                                             ohh shit
565     when robyn posts selfie the world stops for mo...
6849                                 expecting video from
Name: text, dtype: object

In [40]:
x_val_seq = ids_val

In [41]:
x_test_seq = np.load('idsMatrix_test-300.npy')

In [42]:
embedding_matrix = wordVectors
x_test_seq = np.load('idsMatrix_test-300.npy')

In [43]:


seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D

Using TensorFlow backend.


In [44]:


from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

tweet_input = Input(shape=(65,), dtype='int32')

tweet_encoder = Embedding(400000, 300, weights=[embedding_matrix], input_length=65, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 65)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 65, 300)      120000000   input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 64, 100)      60100       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 63, 100)      90100       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [45]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from time import time

filepath="CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

#model.fit(x_train_seq, y_train, batch_size=32, epochs=10,
#                     validation_data=(x_val_seq, y_validation), callbacks = [checkpoint])

model.fit(x_train_seq, y_train, batch_size=32, epochs=10,
                     validation_data=(x_val_seq, y_validation),callbacks=[tbCallBack])

/home/preeti/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 120000000 elements. This may consume a large amount of memory.
  num_elements)


Train on 11253 samples, validate on 1688 samples
Epoch 1/10
11253/11253 [==============================] - 545s 48ms/step - loss: 0.5149 - acc: 0.7511 - val_loss: 0.4628 - val_acc: 0.7838
Epoch 2/10
11253/11253 [==============================] - 540s 48ms/step - loss: 0.3641 - acc: 0.8416 - val_loss: 0.5018 - val_acc: 0.7796
Epoch 3/10
11253/11253 [==============================] - 538s 48ms/step - loss: 0.2014 - acc: 0.9230 - val_loss: 0.6030 - val_acc: 0.7654
Epoch 4/10
11253/11253 [==============================] - 539s 48ms/step - loss: 0.0862 - acc: 0.9701 - val_loss: 0.7890 - val_acc: 0.7737
Epoch 5/10
11253/11253 [==============================] - 540s 48ms/step - loss: 0.0432 - acc: 0.9878 - val_loss: 0.9750 - val_acc: 0.7488
Epoch 6/10
11253/11253 [==============================] - 538s 48ms/step - loss: 0.0330 - acc: 0.9914 - val_loss: 0.9987 - val_acc: 0.7583
Epoch 7/10
11253/11253 [==============================] - 536s 48ms/step - loss: 0.0280 - acc: 0.9918 - val_loss: 1.3

In [47]:
len(x_val_seq)

1688

In [45]:
from keras.models import load_model
from keras.models import load_model
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.01-0.7873.hdf5')


/home/preeti/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 120000000 elements. This may consume a large amount of memory.
  num_elements)


In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
df = pd.read_csv("test_clean.csv")

In [4]:
df.head()

,id,text
0,15923,democrats support antifa muslim brotherhood ms...
1,27014,is revered by conservatives hated by progressi...
2,30530,first it reduces the ca
3,13876,getting the news that she is still up for paro...
4,60133,unity demo to oppose the far right in enough i...


In [5]:
x_test = df.text

In [6]:
test_arr = np.array(x_test)

In [12]:
ids_test = np.zeros((len(test_arr), 65), dtype='int32')
fileCounter = 0

for i in range(len(test_arr)) :
  indexCounter = 0
  line = str(test_arr[i])
  cleanedLine = cleanSentences(line)
  split = cleanedLine.split()
  for word in split:
    try :
      ids_test[fileCounter][indexCounter] = wordsList.index(word)
    except ValueError:
      ids_test[fileCounter][indexCounter] = 399999
      
    indexCounter = indexCounter + 1  
    if indexCounter >= 65:
      break
  fileCounter = fileCounter + 1

In [13]:
np.save('idsMatrix_test-300', ids_test)

In [15]:
embedding_matrix = wordVectors
x_test_seq = np.load('idsMatrix_test-300.npy')

In [109]:
!ls


CNN_best_weights.01-0.6872.hdf5  CNN_best_weights.19-0.7642.hdf5
CNN_best_weights.01-0.6979.hdf5  idsMatrix.npy
CNN_best_weights.01-0.7660.hdf5  idsMatrix_test1.npy
CNN_best_weights.02-0.7032.hdf5  idsMatrix_test.npy
CNN_best_weights.02-0.7305.hdf5  idsMatrix_train1.npy
CNN_best_weights.03-0.7281.hdf5  idsMatrix_train.npy
CNN_best_weights.03-0.7376.hdf5  idsMatrix_validation1.npy
CNN_best_weights.04-0.7417.hdf5  idsMatrix_validation.npy
CNN_best_weights.04-0.7459.hdf5  train_clean.csv
CNN_best_weights.05-0.7482.hdf5  Untitled0.ipynb
CNN_best_weights.05-0.7559.hdf5  wordList.npy
CNN_best_weights.09-0.7565.hdf5  wordVectors.npy
CNN_best_weights.12-0.7618.hdf5


In [16]:
yhat_cnn = loaded_CNN_model.predict(x_test_seq )

In [17]:
yhat_cnn

array([[0.4420126 ],
       [0.3251083 ],
       [0.11227091],
       [0.13798939],
       [0.09334464],
       [0.93603444],
       [0.3231309 ],
       [0.56753045],
       [0.00973487],
       [0.06531769],
       [0.08607245],
       [0.26932386],
       [0.04059109],
       [0.03948797],
       [0.74486595],
       [0.945366  ],
       [0.28234008],
       [0.1255205 ],
       [0.3408002 ],
       [0.40258998],
       [0.29431924],
       [0.33550835],
       [0.4212623 ],
       [0.15073264],
       [0.07246303],
       [0.6234586 ],
       [0.26947033],
       [0.5643202 ],
       [0.32332194],
       [0.8884898 ],
       [0.22593853],
       [0.13418075],
       [0.13731731],
       [0.04265936],
       [0.12999953],
       [0.09240416],
       [0.03433615],
       [0.04270089],
       [0.9404016 ],
       [0.7570864 ],
       [0.81653756],
       [0.3302459 ],
       [0.32071927],
       [0.5834846 ],
       [0.02359405],
       [0.11931337],
       [0.034068  ],
       [0.818

In [18]:
for i in range(len(yhat_cnn)):
    if yhat_cnn[i] < 0.5 :
        yhat_cnn[i] = 0
        yhat_cnn[i] = int(yhat_cnn[i])
    else :
        yhat_cnn[i] = 1
        yhat_cnn[i] = int(yhat_cnn[i])

In [19]:
yhat_cnn

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],

In [20]:
df['cnn'] = yhat_cnn

In [22]:
df.cnn.value_counts()

0.0    684
1.0    176
Name: cnn, dtype: int64

In [23]:
z = {0.0:'NOT',1.0:'OFF'}
df['sentiment'] = df.cnn.map(z)

In [26]:
clean3_df = pd.DataFrame({'id':df.id, 'sentiment':df.sentiment})
clean3_df.head()

,id,sentiment
0,15923,NOT
1,27014,NOT
2,30530,NOT
3,13876,NOT
4,60133,NOT


In [27]:
clean3_df.to_csv('output_cnn_glove_300.csv',header=None ,index=False ,encoding='utf-8')